In [22]:
import argparse
import os
import sys
from PIL import Image, PngImagePlugin
import glob
import numpy as np
np.set_printoptions(threshold=25)
import scipy.fftpack
import pandas as pd
import time
import h5py
start = time.time()


In [ ]:
def get_hash(frame_path, hash_size, high_freq_factor):
    im = Image.open(frame_path).convert("RGB") 
    img_size = hash_size * high_freq_factor
    image = im.convert("L").resize((img_size, img_size), Image.ANTIALIAS)
    pixels = np.asarray(image)
    dct = scipy.fftpack.dct(scipy.fftpack.dct(pixels, axis=0), axis=1)
    dctlowfreq = dct[:hash_size, :hash_size]
    med = np.median(dctlowfreq)
    diff = dctlowfreq > med
    phash = [1 if x == True else 0 for x in diff.flatten()]
    return phash

In [29]:
def write_hashes_to_csv(path_film, wd_hashes, frame_paths, annotation):
    film_name = glob.glob(path_film)[0].split('/')[-1].replace(' ', '-')[0:7]
    hashes = [get_hash(frame, hash_size, high_freq_factor) for frame in frame_paths]
    film_names = [film_name] * len(frame_paths)
    frame_names = [frame.split('/')[-1].split('.')[0] for frame in frame_paths]
    
    data = {'film_name': film_names,
        'frame': frame_names,
       'hash': hashes,
           'annotation': annotation}
    dataset = pd.DataFrame.from_dict(data)
    dataset.to_csv(wd_hashes + '{}-hashes-{}-{}.csv'.format(film_name, hash_size, high_freq_factor), index = False)

In [24]:
hash_size = 15
high_freq_factor = 4

n_film = 3
wd_frames = os.getcwd() + '/films/film_{}/frames/'.format(n_film)
frame_paths = sorted(glob.glob(wd_frames + '*'))

path_film = os.getcwd() + '/films/film_{}/film/*'.format(n_film)
wd_hashes = os.getcwd() + '/films/film_{}/hashes/'.format(n_film)

dataset['annotation'] = np.random.choice(2, len(dataset.hash))

In [26]:
write_hashes_to_csv(path_film, wd_hashes, frame_paths)

NameError: name 'frame_paths' is not defined

In [3]:
def write_elementwise_hdf5(frame_paths, size_hash, hash_size = 15, high_freq_factor = 4, name = ''):
    n_frames = len(frame_paths)
    
    hdf5_store = h5py.File("./phashes{}.hdf5".format('_'+name), "a")
    phashes = hdf5_store.create_dataset("phashes", (n_frames, size_hash), compression="gzip")

    for i in range(0, n_frames):
        frame_path = frame_paths[i]
        phash = get_hash(frame_path, hash_size, high_freq_factor)
        phashes[i] = phash
        
def write_batchwise_hdf5(frame_paths, size_hash, hash_size = 15, high_freq_factor = 4, name = ''):
    n_frames = len(frame_paths)
    
    hdf5_store = h5py.File("./phashes{}.hdf5".format('_'+name), "a")
    phashes = np.zeros((n_frames, size_hash))

    
    for i in range(0, n_frames):
        frame_path = frame_paths[i]
        phash = get_hash(frame_path, hash_size, high_freq_factor)
        phashes[i] = phash

    hdf5_store.create_dataset("phashes", data = phashes, compression="gzip")

def test_writing_speed_hdf5():
    start = time.time()
    write_elementwise(trailer_frames, 122)
    end = time.time()
    print('time for elementwise writing', end-start)
    #time for elementwise writing 42.066429138183594

    start = time.time()
    write_batchwise(trailer_frames, 122)
    end = time.time()
    print('time for batch writing', end-start)
    # time for batch writing 38.23404669761658

def write_hashes_to_hdf5(frame_paths, hash_size = 15, high_freq_factor = 4, name =  ''):
    size_hash = len(get_hash(frame_paths[0], hash_size, high_freq_factor))
    write_elementwise_hdf5(frame_paths, size_hash, hash_size, high_freq_factor, name)

In [15]:
write_hashes_to_hdf5(trailer_frames, name = 'trailer')
write_hashes_to_hdf5(frames, name = 'movie')

time for elementwise writing 43.02076053619385
